<a href="https://colab.research.google.com/github/diegoesquivel09/ejerciciosbsm/blob/main/Eurosourcing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mip

from mip import *


### Definir objeto modelo y los datos


In [2]:
#Crear el modelo

model = Model()

# Datos del problema

mercados = ['Madrid', 'París', 'Berlín', 'Roma', 'Amsterdam']
costo_produccion_local = {'Madrid': 460, 'París': 515, 'Berlín': 505, 'Roma': 480, 'Amsterdam': 490}
demanda = {'Madrid': 420, 'París': 1310, 'Berlín': 1250, 'Roma': 930, 'Amsterdam': 870}
costo_transporte_terrestre = {
    ('Madrid', 'París'): 10, ('Madrid', 'Berlín'): 35, ('Madrid', 'Roma'): 25, ('Madrid', 'Amsterdam'): 25,
    ('París', 'Berlín'): 15, ('París', 'Roma'): 20, ('París', 'Amsterdam'): 10, ('París', 'Madrid'): 10,
    ('Berlín', 'Madrid'): 35,('Berlín', 'París'): 15,('Berlín', 'Roma'): 25, ('Berlín', 'Amsterdam'): 10,
    ('Roma', 'Madrid'): 25, ('Roma', 'París'): 20, ('Roma', 'Berlín'): 25, ('Roma', 'Amsterdam'): 23,
    ('Amsterdam', 'Madrid'): 25, ('Amsterdam', 'París'): 10, ('Amsterdam', 'Berlín'): 10, ('Amsterdam', 'Roma'): 23,
}
costo_transporte_maritimo_china = {'Madrid': 9, 'París': 8.25, 'Berlín': 9, 'Roma': 10, 'Amsterdam': 6}
costo_produccion_china = 440  # Costo unitario de producción en China


#Definir variables

In [8]:
### Definir variables de decisión

produccion_local = {m: model.add_var() for m in mercados}
produccion_china = model.add_var()

#Restricciones de demanda

In [10]:
model.add_constr(
    xsum(produccion_local[m] for m in mercados)
    + produccion_china + xsum(costo_transporte_maritimo_china[m] * produccion_china for m in mercados)
    + xsum(costo_transporte_terrestre.get((m, otro_m), 0) * produccion_local[otro_m] for m in mercados for otro_m in mercados if m != 'China')
    == xsum(demanda[m] + xsum(costo_transporte_terrestre.get((otro_m, m), 0) * produccion_local[m] for otro_m in mercados if otro_m != 'China') for m in mercados)
  )



# Función Objetivo

In [11]:
model.objective= minimize(
    xsum(produccion_local[m] * costo_produccion_local[m] for m in mercados)
    + produccion_china * costo_produccion_china
    + xsum(costo_transporte_maritimo_china[m] * produccion_china for m in mercados)
    +xsum(costo_transporte_terrestre.get((m, otro_m),0) * produccion_local[otro_m] for m in mercados for otro_m in mercados)
)



Resolver el model

In [14]:
status= model.optimize()
print("Estado:", status)

print("Producción loca len cada mercado:")
for m in mercados:
  print(f"{m}: {produccion_local[m].x}")

print(f"Producción en China: {produccion_china.x}")
print("Costo total de producción y transporte:",  model.objective_value)

Estado: OptimizationStatus.OPTIMAL
Producción loca len cada mercado:
Madrid: 0.0
París: 0.0
Berlín: 0.0
Roma: 0.0
Amsterdam: 0.0
Producción en China: 110.52023121387283
Costo total de producción y transporte: 53298.38150289017
